# Use your deployed model from a notebook using Langchain

## Background
This tutorial is designed to give you the basics of using langchain to work with Large Language Models (LLMs) for document summarization and basic chat bot functionality. You could take what we have here to build a front end application using something like streamlit, or other further iterations.

### Install Packages

In [12]:
! pip install "langchain" "faiss-cpu"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 85.2 MB/s eta 0:00:0000:0100:01


### Load Libraries

In [12]:
import os
from langchain.schema import HumanMessage
from langchain.vectorstores import FAISS
from langchain.document_loaders import WebBaseLoader
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import RetrievalQA


os.environ["AZURE_OPENAI_API_KEY"] = "01f0a2f985f44fe182f226a8b33aa03a"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://nih-openai-poc.openai.azure.com/"


Try loading in your deployed model and running a simple prompt

In [2]:
llm = AzureChatOpenAI(
    openai_api_version="2023-05-15",
    #openai_api_base="https://nih-openai-poc.azure.openai.com/openai/deployments/nih-openai-poc-35-0301",
    azure_endpoint = "https://nih-openai-poc.openai.azure.com/openai/deployments/nih-openai-poc-35-0301/chat/completions?api-version=2023-07-01-preview",
    openai_api_key="01f0a2f985f44fe182f226a8b33aa03a"
)

Run the prompt against the LLM

In [3]:
message = HumanMessage(
    content="Translate this sentence from English to French: Why all the hype about Generative AI?"
)
llm([message])

AIMessage(content="Pourquoi tout ce battage médiatique autour de l'IA générative ?")

### Use the LLM to summarize a scientific document

Now let's load in a scientific document to run a query against. Read more about document loaders from langchain [here](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf). Note that we are both loading, and splitting our document. You can read more about the default document chunking/splitting procedures [here](https://python.langchain.com/docs/modules/data_connection/document_transformers/#get-started-with-text-splitters).

In [4]:
loader = WebBaseLoader("https://pubmed.ncbi.nlm.nih.gov/37883540/")
pages = loader.load_and_split()

Define that we want to use [stuffing](https://python.langchain.com/docs/modules/chains/document/stuff) to summarize the document.

In [5]:
chain = load_summarize_chain(llm, chain_type="stuff")

chain.run(pages)



'A study published in Science has found that wild chimpanzees also experience menopause, a phenomenon previously thought to be unique to human females and a few species of toothed whales. The research showed that post-reproductive life spans exist among female chimpanzees. Post-reproductive representation was found to be 0.195, indicating that a female who reached adulthood could expect to live about one-fifth of her adult life in a post-reproductive state, around half as long as human hunter-gatherers. The implications for the understanding of the evolution of post-reproductive life spans are discussed.'

### Use the LLM and Vector Stores to summarize a scientific document

We can even try embed our splitted document and store them into a vectore store. Embeddings convert our text into a numerical vector the numbers that each text contains represents how similar certain texts are, this helps our model find texts that are related or similar to the text in our question. Here we are utilizing the OpenAi embeddings model **text-embedding-ada-002**.

In [7]:
embeddings = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002",
    chunk_size=10, #processing our chunks in batches of 10
)

Now that we have set up our embeddings model we can now set up our vector store which automatically embed our split document. Here we are using the FAISS vector store to store our split document and its embeddings.

In [8]:
db = FAISS.from_documents(pages, embeddings)

We will treat our vector store as a retriever to allow our model to gather the splited parts of our document that are related to the question we ask. 

In [15]:
llm_chain = RetrievalQA.from_chain_type(llm=llm, retriever = db.as_retriever(), chain_type="stuff")

Now we'll ask our model a question about our document!

In [17]:
question = "What did the study find about chimpanzees?"

llm_chain.run(question)

'The study found that post-reproductive life exists among wild chimpanzees in the Ngogo community of Kibale National Park, Uganda. Post-reproductive females exhibited hormonal signatures of menopause, including sharply increasing gonadotropins after age 50. The study also discusses whether post-reproductive life spans in wild chimpanzees occur only rarely, as a short-term response to favorable ecological conditions, or instead are an evolved species-typical trait as well as the implications of these alternatives for our understanding of the evolution of post-reproductive life spans.'

The next step is to create a chat bot via embeddings using Azure AI Search. That's in the next tutorial!